# About
In recent years, Cloud Model 1 (CM1; http://www2.mmm.ucar.edu/people/bryan/cm1/) has become a very popular tool for performing idealized studies of atmospheric phenomena. There exists very little support for computing trajectories using CM1 output, which are usually necessary to understand the processes of the atmospheric phenomena of interest. Natively, CM1 only supports 'online' forward trajectories in 2D simulations and in 3D simulation without terrain. I wrote this script because there are no adequate tools available to compute highly customizable 'offline' trajectories in simulations with terrain. This script is intended to be easily customizable.

Notes:

* Can compute backward or forward trajectories (Default is backward, but can be forward with simple changes to "Calculate Trajectories" block)
* Written to work with 3D model output (can be modified to work with 2D output)
* Will work with or without terrain
* Initial location, number, and density of parcels can be easily specified in "Initialize Parcels" block
* Uses xarray and Dask to distribute memory and calculation across multiple processors
* With modifications, can be used with WRF output (several others have already done so)
* Comments that say "set by user" are specific to model output and desired trajectories

# Load Modules

In [ ]:
import os, sys
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import xarray as xr
from scipy import interpolate
import time
import scipy.ndimage

%config IPCompleter.greedy=True

# Read in CM1 Output

* User must insert path to data
    * If model output is one file use ***xr.open_dataset***
    * If model output is in multiple files use ***xr.openmfdataset***


In [ ]:
#Use xarray to open model output and specify chunking if data set is large (set by user)
i_chunk = 10
k_chunk = 1
ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/steenburgh-group8/tom/cm1/output/12ms_2000m_tug.nc', chunks={'nk': k_chunk})

#Get model output dimensions
num_x = ds.nx #Number of gridpoints in x
num_y = ds.ny #Number of gridpoints in y
num_z = ds.nz #Number of gridpoints in z

x = np.arange(0,num_x,1)
y = np.arange(0,num_y,1)
z = np.arange(0,num_z,1)

# Create Dask Cluster

In [ ]:
#Option to use multiple processors and threads (set by user)
from dask.distributed import Client, LocalCluster
c = LocalCluster(n_workers=10, threads_per_worker=1)
client = Client(c)
client

# Initialize Parcels

User must enter desired trajectory characteristics

In [ ]:
#Number of parcels in x (set by user) (every 50th grdpt)
num_seeds_x = np.int(2100/25)

#Number of parcels in y (set by user) (every 50th grdpt)
num_seeds_y = np.int(num_y/25)

#Number of time steps to run trajectories forward (set by user) 
time_steps = 500
incre = 1

#Time step to start forwards trajectories at (set by user) 
start_time_step = 1

#Variable to record at each parcel's location throughout trajectory (code can be easily modified to add more; set by user) 
var_name1 = 'th'

#Set as 'Y' or 'N' for 'yes' or 'no' if the u, v, and w model output is on the staggered grid 
#(unless you have interpolated u, v, and w to the scalar grid, they are most likely on the staggered grid (set by user)
staggered = 'N'

**Model output info**

In [ ]:
#Horizontal resolution of model output (meters)
hor_resolution = (ds.xf[1].values-ds.xf[0].values)*1000

#Vertical resolution of model output (meters). Changes in x and y, if there is terrain, and z, if grid is stretched.
try:
    vert_resolution = ds.zh[0,1:,:,:].values-ds.zh[0,:-1,:,:].values
    print('Output has terrain')
except:
    vert_res1d = (ds.z[1:].values-ds.z[:-1].values)*1000
    vert_res2d = np.repeat(vert_res1d,ds.ny, axis = 0).reshape(ds.nz-1, ds.ny)
    vert_resolution = np.repeat(vert_res2d,ds.nx, axis = 0).reshape(ds.nz-1, ds.ny, ds.nx)
    print('Output does not have terrain')
                  
#Model output time step length (seconds)
time_step_length = (ds.time[1].values - ds.time[0].values)/np.timedelta64(1, 's')*incre

**Create empty arrays to store x, y, and z positions of parcels**

In [ ]:
xpos = np.zeros((time_steps, num_seeds_y, num_seeds_x)) #x-location (grid points on staggered grid)
ypos = np.zeros((time_steps, num_seeds_y, num_seeds_x)) #y-location (grid points on staggered grid)
zpos = np.zeros((time_steps, num_seeds_y, num_seeds_x)) #z-location (grid points on staggered grid)
zpos_heightASL = np.zeros((time_steps, num_seeds_y, num_seeds_x)) #Height above sea level (meters)
zpos_vert_res = np.zeros((time_steps, num_seeds_y, num_seeds_x)) #Vertical grid spacing at parcel location (meters)
variable1 = np.zeros((time_steps, num_seeds_y, num_seeds_x)) #User specified variable to track

**Initial location of parcels in gridpoints, specifically on the scalar grid (set by user). Initializes an array of parcels in the the y-z domain (modification necessary for x-dimension or 3D array of parcels)**

In [ ]:
#x-position
for i in range(num_seeds_x):
    xpos[0,:,i] = 25*i+1500 #This example initializes all seeds at same x-position (1000th x-grpt, set by user)

#y-position   
for i in range(num_seeds_y):
    ypos[0,i,:] = 25*i #This example initializes seeds evenly in y-dimension (0th, 4th, 8th, etc. y-grpt; set by user)

#z-position
zpos[0,:,:] = 0 #This example initializes seeds evenly starting in z-dimension (0th, 1st, 2nd, etc., z-grpt; set by user)

## Determine Initial Height of Parcels Above Sea Level
Use the height of the models levels (meters above sea level) to convert from terrain following grid points to height above seal level.

In [ ]:
#Get height of surface
try:
    zs = ds.zs[0,:,:].values
    print('Output has terrain')
except: 
    zs = np.zeros((ds.ny,ds.nx))
    print('Output does not have terrain')


#Get height of vertical coordinates (scalar grid)
try:
    zh = ds.zh[0,:,:,:].values
    print('Output has terrain')
except:
    zh1d = (ds.z[:].values)*1000
    zh2d = np.repeat(zh1d,ds.ny, axis = 0).reshape(ds.nz, ds.ny)
    zh = np.repeat(zh2d,ds.nx, axis = 0).reshape(ds.nz, ds.ny, ds.nx)
    print('Output does not have terrain')

#Create list of initial coordinates to get height
xloc = (xpos[0,:,:]).flatten()
yloc = (ypos[0,:,:]).flatten()
zloc = (zpos[0,:,:]).flatten()
coord_height = []
for i in range(len(xloc)):
    coord_height.append((zloc[i], yloc[i], xloc[i]))

#Get the actual inital height of the parcels in meters above sea level
zpos_heightASL[0,:,:] = np.reshape(interpolate.interpn((z,y,x), zh, coord_height, method='linear', bounds_error=False, fill_value= 0), (num_seeds_y, num_seeds_x))

# Calculate Trajectories
Unless user is changing trajectories from backwards to forwards, nothing should be changed here.

In [ ]:
#Inital locations for loop
xpos_init = np.array(xpos[0,:,0])
ypos_init = np.array(ypos[0,:,0])
zpos_init = np.array(zpos[0,:,0])
zpos_init = np.array(zpos[0,:,0])
zpos_heightASL_init = np.array(zpos_heightASL[0,:,0])



#Loop over all time steps and compute trajectory
for t in range(time_steps-1):
    
    #Get new array lengths because we append
    num_seeds_x = len(xpos[0,0,:])
    num_seeds_y = len(xpos[0,:,0])
    
    
    
    start = time.time() #Timer
    
    xmin = np.int(np.nanmin(xpos[t,:,:])-2)
    xmin = 0 if xmin < 0 else xmin
    
    xmax = np.int(np.nanmax(xpos[t,:,:])+2)
    xmax = ds.nx if xmax > ds.nx else xmax
    
    ymin = np.int(np.nanmin(ypos[t,:,:])-2)
    ymin = 0 if ymin < 0 else ymin
    
    ymax = np.int(np.nanmax(ypos[t,:,:])+2)
    ymax = ds.ny if ymax > ds.ny else ymax
    
    zmin = np.int(np.nanmin(zpos[t,:,:])-2)
    zmin = 0 if zmin < 0 else zmin
    
    zmax = np.int(np.nanmax(zpos[t,:,:])+2)
    zmax = ds.nz if zmax > ds.nz else zmax
    
    x_fast = np.arange(0,xmax-xmin)
    y_fast = np.arange(0,ymax-ymin)
    z_fast = np.arange(0,zmax-zmin)
    
    #Get model data
    t = np.int(t * incre)
    
    startf = time.time() #Timer
    u = ds.uinterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    v = ds.vinterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    w = ds.winterp[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    var1 = getattr(ds,var_name1)[start_time_step+t,zmin:zmax,ymin:ymax,xmin:xmax].values
    stopf = time.time()#Timer
    print("Load data {:01d} took {:.2f} seconds".format(t, stopf-startf))
    
    t = np.int(t/incre) 
    
    #Smooth data
    smooth = 20
    u = np.copy(scipy.ndimage.filters.uniform_filter(u, smooth))
    v = np.copy(scipy.ndimage.filters.uniform_filter(v, smooth))
    w = np.copy(scipy.ndimage.filters.uniform_filter(w, smooth))
    var1 = np.copy(scipy.ndimage.filters.uniform_filter(var1, smooth))
    
    
        
    ############## Generate coordinates for interpolations ###############

    #x, y, and z on staggered and scalar grids
    xloc = np.copy(xpos[t,:,:]).flatten()-xmin
    xloc_stag = np.copy(xpos[t,:,:]+0.5).flatten()-xmin
    yloc = np.copy(ypos[t,:,:]).flatten()-ymin
    yloc_stag = np.copy(ypos[t,:,:]+0.5).flatten()-ymin
    zloc = np.copy(zpos[t,:,:]).flatten()-zmin
    zloc_stag = np.copy(zpos[t,:,:]+0.5).flatten()-zmin

    #If u, v, and w are staggered, generate three staggered sets of coordinates:
    #    1) u-grid (staggered in x)
    #    2) v-grid (staggered in y)
    #    3) w-grid (staggered in z)
    
    if staggered == 'Y':
        coord_u = []
        coord_v = []
        coord_w = []
        for i in range(len(xloc)):
            coord_u.append((zloc[i], yloc[i], xloc_stag[i])) 
            coord_v.append((zloc[i], yloc_stag[i], xloc[i])) 
            coord_w.append((zloc_stag[i], yloc[i], xloc[i])) 
    
    #If not, generate scalar coordinates
    else: 
        coord_u = []
        coord_v = []
        coord_w = []
        for i in range(len(xloc)):
            coord_u.append((zloc[i], yloc[i], xloc[i])) 
            coord_v.append((zloc[i], yloc[i], xloc[i])) 
            coord_w.append((zloc[i], yloc[i], xloc[i])) 
    
    #Scalar coordinates for all other variables
    coord = []
    coord_fast = []
    for i in range(len(xloc)):
        coord.append((zloc[i]+zmin, yloc[i]+ymin, xloc[i]+xmin)) 
        coord_fast.append((zloc[i], yloc[i], xloc[i])) 
    
    ##########################################################################################################   
    ########################## Integrate to determine parcel's new location ##################################
    ##########################################################################################################   

    
    #########################   Calc new xpos in grdpts above surface  #######################################
    xpos[t+1,:,:] = xpos[t,:,:] + np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), u, coord_u, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_y, num_seeds_x))

    #########################   Calc new ypos in grdpts above surface  #######################################
    ypos[t+1,:,:] = ypos[t,:,:] + np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), v, coord_v, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_y, num_seeds_x))

    #########################   Calc new zpos in meters above sea level ######################################
    zpos_heightASL[t+1,:,:] = zpos_heightASL[t,:,:] + np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), w, coord_w, method='linear', bounds_error=False, fill_value= 0)*time_step_length, (num_seeds_y, num_seeds_x))

    ############# Convert zpos from meters above sea level to gridpts abve surface for interpolation #########
    #Get vertical grid spacing at each parcel's location
    zpos_vert_res[t,:,:] = np.reshape(interpolate.interpn((z[:-1],y,x), vert_resolution, coord, method='linear', bounds_error=False, fill_value= np.nan), (num_seeds_y, num_seeds_x))

    
    #Calculate change in surface height and change in parcel height
    xloc = np.copy(xpos[t,:,:]).flatten()
    yloc = np.copy(ypos[t,:,:]).flatten()
    coord_zs1 = []
    for i in range(len(xloc)):
        coord_zs1.append((yloc[i], xloc[i]))
        
    xloc = np.copy(xpos[t+1,:,:]).flatten()
    yloc = np.copy(ypos[t+1,:,:]).flatten()
    coord_zs2 = []
    for i in range(len(xloc)):
        coord_zs2.append((yloc[i], xloc[i]))
    
    #Change in sruface height over last timestep
    zs1 = interpolate.interpn((y,x), zs, coord_zs1, method='linear', bounds_error=False, fill_value= np.nan)
    zs2 = interpolate.interpn((y,x), zs, coord_zs2, method='linear', bounds_error=False, fill_value= np.nan)
    zs_change = zs2-zs1
    
    #Change in parcel height over last times step
    zpos_heightASL_change = zpos_heightASL[t+1,:,:].flatten()-zpos_heightASL[t,:,:].flatten()
    
    #Calculate zpos in grdpts above surface
    zpos[t+1,:,:] = zpos[t,:,:] + np.reshape((zpos_heightASL_change - zs_change)/zpos_vert_res[t,:,:].flatten(), (num_seeds_y, num_seeds_x))
    ##########################################################################################################

    
    
    #Prevent parcels from going into the ground
    zpos = zpos.clip(min=0)
    zpos_heightASL = zpos_heightASL.clip(min=0)
    
    #Calculate value of variable at each parcel's location
    variable1[t,:,:] = np.reshape(interpolate.interpn((z_fast,y_fast,x_fast), var1, coord_fast, method = 'linear', bounds_error=False, fill_value= np.nan), (num_seeds_y, num_seeds_x))  
    
    
    ##################   Initialize new set of parcels #######################
    #Initalize new set of parcels to enter the domain every other time step. #
    #Idea is to create new 2d array and append it to the originals          ##
    ##########################################################################
    if t > 0:
        if t % 3 == 0: #Only send in new parcels every other timestep
            #Arrays for new set of trajectories as that enter the domain
            xpos_new = np.zeros((time_steps, num_seeds_y, 1))
            ypos_new = np.zeros((time_steps, num_seeds_y, 1))
            zpos_new = np.zeros((time_steps, num_seeds_y, 1))
            zpos_heightASL_new = np.zeros((time_steps, num_seeds_y, 1))
            zpos_vert_res_new = np.zeros((time_steps, num_seeds_y, 1))
            variable1_new = np.zeros((time_steps, num_seeds_y, 1))


            #Location of new parcels that enter domain
            xpos_new[t+1,:,0] = np.array(xpos_init)
            print(xpos_new[t+1,:,0])
            ypos_new[t+1,:,0] = np.array(ypos_init)
            print(ypos_new[t+1,:,0])
            zpos_new[t+1,:,0] = np.array(zpos_init)
            print(zpos_new[t+1,:,0])
            zpos_heightASL_new[t+1,:,0] = np.array(zpos_heightASL_init)
            print(zpos_heightASL_new[t+1,:,0])

            #Set past time steps to nan
            xpos_new[:t+1,:,:] = np.nan
            ypos_new[:t+1,:,:] = np.nan
            zpos_new[:t+1,:,:] = np.nan
            zpos_heightASL_new[:t+1,:,:] = np.nan


            #Append new arrays to old arrays
            xpos = np.append(xpos_new, xpos, axis = 2)
            ypos = np.append(ypos_new, ypos, axis = 2)
            zpos = np.append(zpos_new, zpos, axis = 2)
            zpos_heightASL = np.append(zpos_heightASL_new, zpos_heightASL, axis = 2)
            zpos_vert_res = np.append(zpos_vert_res_new, zpos_vert_res, axis = 2)
            variable1 = np.append(variable1_new, variable1, axis = 2)


    

    #Timer
    stop = time.time()
    print("Integration {:01d} took {:.2f} seconds".format(t, stop-start))

Get variable data for final time step

In [ ]:
t = time_steps-1
var1 = getattr(ds,var_name1)[start_time_step-t,:,:,:].values

#Get get x, y, and z positions from scalar grid
xloc = np.copy(xpos[t,:,:]-0.5).flatten()
yloc = np.copy(ypos[t,:,:]-0.5).flatten()
zloc = np.copy(zpos[t,:,:]-0.5).flatten()
coord = []
for i in range(len(xloc)):
    coord.append((zloc[i], yloc[i], xloc[i])) 

#Variables
variable1[t,:,:] = np.reshape(interpolate.interpn((z,y,x), var1, coord, method = 'linear', bounds_error=False, fill_value= np.nan), (num_seeds_y, num_seeds_x))

# Save Trajectory Data
The x, y, and z positions and user-specified variable values are saved in 3D numpy arrays. The first dimension is time and the other two are the positions and values of variables of all the parcels at that specifc time.

In [ ]:
np.save('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/cm1/trajectory_data/xpos_2000m_tug_smooth', xpos)
np.save('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/cm1/trajectory_data/ypos_2000m_tug_smooth', ypos)
np.save('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/cm1/trajectory_data/zpos_2000m_tug_smooth', zpos_heightASL)
np.save('/uufs/chpc.utah.edu/common/home/u1013082/lake_effect/cm1/trajectory_data/%s_2000m_tug_smooth' %var_name1, variable1)